## Dart 페이지 크롤링하기

In [234]:
# 필요한 패키지들
# 1) 통신 : urllib, requests

import requests

#2) html -> tag 중심의 언어
    # html의 테그를 중심으로 정보 추출 : BeautifulSoup
from bs4 import BeautifulSoup

#3) 데이터 처리
import pandas as pd

#4) + 정규식처리, 속도제어(지연 처리)
import re
import time
from datetime import datetime

In [2]:
# step1 ) 내가 요청하고자 하는 URL을 완성해야 함!
# -> API는 문서보고 하면 되는데
# -> 일반 사이트는 내가 열심히 필요한 정보를 요청하는 URL 탐색

In [3]:
# 예시 날짜 : 2024.03.05
# -> 698건의 공시 정보를 요청하려고 함.
# -> 1페이지가 아니라 여러 페이지를 롤링해야 함.

date= "2024.03.05" #<-selectDate=
page = "1" #<-currentPage

url = f"https://dart.fss.or.kr/dsac001/mainAll.do?selectDate={date}&sort=&series=&mdayCnt=0&currentPage={page}"
url


'https://dart.fss.or.kr/dsac001/mainAll.do?selectDate=2024.03.05&sort=&series=&mdayCnt=0&currentPage=1'

In [4]:
#step2 ) requests 패키지로 요청
# -> 어제는 FM 적으로 params를 dict 로 만들어서 요청
# -> 이번에는 urlopen처럼 full url 직접 던질 때
res = requests.get(url)
res

<Response [200]>

In [5]:
# 참고) request 패키지로 받은 정보를 열어보는 방식 2가지
# 1) res.text : 알아서 자기가 인코딩/ 디코딩을 처리를 함
#             특별하게 신경쓸 부분이 거의 없음!
# 2) res.content: 이미지, 동영상 --> 바이트값으로 들어옴.
# -> 일반적으로 특수한(이미지, 동영상), text를 주로 활용

In [8]:
res.text

'<!DOCTYPE html>\r\n\r\n\r\n\r\n\n\n\n\n\n\n\n\r\n<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="ko" lang="ko">\r\n\t<head>\r\n\t\t<title>최근공시</title>\r\n\t\t<meta charset="UTF-8">\r\n\t\t<meta id="Viewport" name="viewport" content="width=device-width,initial-scale=1.0,minimum-scale=1.0,maximum-scale=1.0,user-scalable=yes">\r\n\t\t<meta http-equiv="X-UA-Compatible" content="ie=edge">\r\n\t\t\r\n\r\n\r\n\t\r\n<script type="text/javascript" src="/resource/js/jquery-3.3.1.min.js"></script>\r\n\r\n<link rel="stylesheet" href="/js/jquery-ui/jquery-ui.min.css">\r\n<script type="text/javascript" src="/js/jquery-ui/jquery-ui.min.js"></script>\r\n\r\n<!-- 2011.11.01 ext 2.3 -->\r\n<!--[if lte IE 8]><link rel="stylesheet" type="text/css" href="/js/ext-main/resources/css/ext-all-ie8.css" /><![endif]-->\r\n<script type="text/javascript" src="/js/ext-main/adapter/ext/ext-base.js"></script>\r\n<script type="text/javascript" src="/js/ext-main/ext-all.js"></script>\r\n\r\n<!-- x-xeries js librar

In [9]:
# 위의 요청한 웹사이트의 내용을 필요한 정보에 쉽게 
# tag 중심으로 접근하기 위해서는 beautifulSoup 패키지로 변환!
# -> 정규식으로 모든 텍스트로 필요한 정보를 접근하는 게 아니라
#     tag 중심으로 정보 접근!

soup = BeautifulSoup(res.text, "html.parser")
soup

<!DOCTYPE html>

<html lang="ko" xml:lang="ko" xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>최근공시</title>
<meta charset="utf-8"/>
<meta content="width=device-width,initial-scale=1.0,minimum-scale=1.0,maximum-scale=1.0,user-scalable=yes" id="Viewport" name="viewport"/>
<meta content="ie=edge" http-equiv="X-UA-Compatible"/>
<script src="/resource/js/jquery-3.3.1.min.js" type="text/javascript"></script>
<link href="/js/jquery-ui/jquery-ui.min.css" rel="stylesheet"/>
<script src="/js/jquery-ui/jquery-ui.min.js" type="text/javascript"></script>
<!-- 2011.11.01 ext 2.3 -->
<!--[if lte IE 8]><link rel="stylesheet" type="text/css" href="/js/ext-main/resources/css/ext-all-ie8.css" /><![endif]-->
<script src="/js/ext-main/adapter/ext/ext-base.js" type="text/javascript"></script>
<script src="/js/ext-main/ext-all.js" type="text/javascript"></script>
<!-- x-xeries js libraries  -->
<script src="/js/xjs.js?ver=1.18" type="text/javascript"></script>
<!-- application js libraries -->
<script sr

In [10]:
# -> 웹 페이지의 내용을 파이썬으로 땡겨옴!
# 이제 나의 필요한 정보를 테그 중심으로 접근

# -> xml 은 단순 tag값만 알면 정보 찾는데 문제가 없는데, 
# -> html은 tag안에 많은 표현/ 출력에 관련도니 속성들이 존재
# 단순 tag 하나만으로는 접근하기 애매함 : tag + tag 의 속성들을 같이 활용해서 접근

In [11]:
# 방법 1) dict로 상세 속성을 지정하는 방식
len(soup.find_all("div"))

58

In [12]:
soup.find_all("div")[0] #롤링해서 하나하나 찾는 건 비효율적!!!

<div class="loadingMask" id="loadingMask" style="display:none"></div>

In [14]:
soup.find_all("div", {"class":"headerTop"})

[<div class="headerTop">
 <h1 class="logo" title="DART시스템 "><a href="/main.do">홈으로 가기</a></h1>
 </div>]

In [15]:
# 방법2) tag 중에서 bs4 가 이미 파라미터로 만들어둔 것!
#     ex) class 파라미터의 경우에는 : class_
soup.find_all("div", class_ = "headerTop")

[<div class="headerTop">
 <h1 class="logo" title="DART시스템 "><a href="/main.do">홈으로 가기</a></h1>
 </div>]

In [31]:
# Q) 전체 그 날의 공시 자료 수 체크
#     -> 내가 몇 개의 페이지를 돌려야 할지(그냥 1페이지부터 쭉 돌리면서, 체크를 하는 방법도 있음)
#     -> 정규식 같은 것도 쓰고, 뭔가 규칙을 찾아보기 위함(수업적 내용)
#       테이블 아래 [1/7] [총 698건] 에서 찾아보도록 하겠음.  

temp = soup.find_all("div", {"class":"pageInfo"})[0].text
temp

'[1/7] [총 698건]'

In [26]:
# + 필요한 부분 정규식!
# -> 정규식에서 특정 문자 패턴을 찾는 것 : re.findall(패턴, 어디서)
# -> 정규식에서 특정 문자 패턴을 변경/ 제거 : re.sub(패턴, 무엇으로 변경, 어디서)


In [36]:
# try1 ) 찾을 패턴 : 숫자들 + 건
temp = re.findall(r"\d+건", temp)[0] # 698건
temp = re.sub(r"건","",temp)
temp

'698'

In [39]:
# 페이지 계산 : 1페이지 최대 100건
# 100으로 나눠서 몫을 올림

if int(temp) % 100 == 0:
    tot_page= int(temp) //100
else :
    tot_pate = int(temp) //100 + 1

In [59]:
# try2) 이번에는 정보에서 /???] --? ???페이지 정보추출

temp = soup.find_all("div", {"class","pageInfo"})[0].text
temp1 = re.findall(r"/\d+\]",temp)[0] # re.findall(r"[0-9]+",temp)
temp1 = re.sub(r"\/|\]","",temp1)
temp1

'7'

In [60]:
# 필요한 정보(공시관련)들을 추출해보자.
# Q) 1개 공시 정보에 대한 tr 테그를 찾아보자
soup.find_all("tr") # 테이블 head(1), body(100) = 101개

[<tr>
 <th scope="row"><label for="inpSample00">시간</label></th>
 <th scope="row"><label for="inpSample00">공시대상회사</label></th>
 <th scope="row"><label for="inpSample00">보고서명</label></th>
 <th scope="row"><label for="inpSample00">제출인</label></th>
 <th scope="row"><label for="inpSample00">접수일자</label></th>
 <th scope="row"><label for="inpSample00">비고</label></th>
 </tr>,
 <tr>
 <td>
 								
 								
 								
 								19:29
 							</td>
 <td class="tL">
 <span class="innerWrap">
 <span class="tagCom_kosdaq" style="cursor:default" title="코스닥시장">코</span>
 <a href="javascript:openCorpInfoNew('01375822', 'winCorpInfo', '/dsae001/selectPopup.ax');" title="에이프릴바이오 기업개황 새창">
 										에이프릴바이오
 									</a>
 </span>
 </td>
 <td class="tL">
 <a href="/dsaf001/main.do?rcpNo=20240305901186" id="r_20240305901186" onclick="openReportViewer('20240305901186'); return false;" title="매출액또는손익구조30%(대규모법인은15%)이상변동 공시뷰어 새창"><span class="txtCB" title="본 보고서명으로 이미 제출된 보고서의 기재내용이 변경되어 제출된 것임">[기재정정]</span>

In [64]:
# -> 정확하게 공시 정보들에 대해서 접근을 하기 위해서
#     tbody 속에 있는 tr로 접근하는 게 아니라 좀 더 나아보임!
# -> 모든 tr 나오세요가 아니라 thead에 tr 나오니..
# -> tbody 속에 있는 tr만 다 나오세요! -> 공시들만


soup.find("tbody").find_all("tr")[0]

<tr>
<td>
								
								
								
								19:29
							</td>
<td class="tL">
<span class="innerWrap">
<span class="tagCom_kosdaq" style="cursor:default" title="코스닥시장">코</span>
<a href="javascript:openCorpInfoNew('01375822', 'winCorpInfo', '/dsae001/selectPopup.ax');" title="에이프릴바이오 기업개황 새창">
										에이프릴바이오
									</a>
</span>
</td>
<td class="tL">
<a href="/dsaf001/main.do?rcpNo=20240305901186" id="r_20240305901186" onclick="openReportViewer('20240305901186'); return false;" title="매출액또는손익구조30%(대규모법인은15%)이상변동 공시뷰어 새창"><span class="txtCB" title="본 보고서명으로 이미 제출된 보고서의 기재내용이 변경되어 제출된 것임">[기재정정]</span>매출액또는손익구조30%(대규모법인은15%)이상변동
									
		  							
								</a>
</td>
<td class="tL ellipsis" title="에이프릴바이오">에이프릴바이오</td>
<td>2024.03.05</td>
<td><span class="tagCom_kosdaq_other" style="cursor:default" title="본 공시사항은 한국거래소 코스닥시장본부 소관임">코</span></td>
</tr>

In [77]:
idx = 0
temp = soup.find("tbody").find_all("tr")[idx]
temp

# 1) 1번 공시의 제출 시간 
# -> 안 좋은 공시 : 주말, 금요일 장 끝나고...

temp.find_all("td")[0].text # 원하는 정보는 있지만 공백 문자들이 존재 strip(): 양쪽 or 정규식
temp.find_all("td")[0].text.strip() # 양쪽에 공백만 날려줌 -> 정규식으로 공백처리를 해야함.
# re.sub(r"\r|\n|\t\s","",str(temp))

'19:29'

In [95]:
# Q) 1번 공시를 제출한 회사가 속한 시장을 보자

temp.find_all("td")[1].find_all("span")[1].get("title")



'코스닥시장'

In [97]:
# Q) 1번 공시 대상인 회사의 이름을 출력!
temp.find_all("td")[1].text.strip()
#--> 처리할 수는 있는데 정확히 타겟팅은 안 됨

'코\n\r\n\t\t\t\t\t\t\t\t\t\t에이프릴바이오'

In [104]:
temp.find_all("td")[1].find("a").text.strip()

'에이프릴바이오'

In [121]:
temp.find_all("td")[1]

<td class="tL">
<span class="innerWrap">
<span class="tagCom_kosdaq" style="cursor:default" title="코스닥시장">코</span>
<a href="javascript:openCorpInfoNew('01375822', 'winCorpInfo', '/dsae001/selectPopup.ax');" title="에이프릴바이오 기업개황 새창">
										에이프릴바이오
									</a>
</span>
</td>

In [118]:
# Q) 1번 공시의 회사의 코드값을 출력해보세요.
temp.find_all("td")[1].find("a").get("href")

"javascript:openCorpInfoNew('01375822', 'winCorpInfo', '/dsae001/selectPopup.ax');"

In [120]:
re.findall(r"\d{8}",temp.find_all("td")[1].find("a").get("href"))

['01375822']

In [136]:
# Q) 1번 공시의 고유값 rcpNo 
re.findall(r"\d{14}",temp.find_all("td")[2].find("a").get("href"))

# Q) 1번 공시의 공시 이름 출력

['20240305901186']

In [139]:
temp.find_all("td")[2].find("a").text.strip()

'[기재정정]매출액또는손익구조30%(대규모법인은15%)이상변동'

In [204]:
# Q) 1번 공시에 대한 요청 일자
temp.find_all("td")[4].text


'2024.03.06'

In [212]:
# Q1) for문을 이용해서 지금 페이지의 공시 정보를 출력
# -> 제출 시간, 회사가 속한 시장(fullname), 회사이름, 회사 코드, 공시이름, 공시의 고유값, 요청일자
cnt= len(soup.find("tbody").find_all("tr"))
         
for idx in range(cnt):
    temp = soup.find("tbody").find_all("tr")[idx]
    sub_time = temp.find_all("td")[0].text.strip()
    market = temp.find_all("td")[1].find_all("span")[1].get("title")
    cName = temp.find_all("td")[3].text
    code = re.findall(r"\d{8}",temp.find_all("td")[1].find("a").get("href"))[0]
    tName= re.sub(r"\t|\n","",temp.find_all("td")[2].text.strip())
    rcpNo= re.findall(r"\d{14}",temp.find_all("td")[2].find("a").get("href"))[0]
    ord_date = temp.find_all("td")[4].text
    print("제출시간", sub_time)
    print("시장", market)
    print("회사이름", cName)
    print("회사 코드", code)
    print("공시이름", tName)
    print("고유값", rcpNo)
    print("요청일자", ord_date)
    print("#"*40)

제출시간 19:29
시장 코스닥시장
회사이름 에이프릴바이오
회사 코드 01375822
공시이름 [기재정정]매출액또는손익구조30%(대규모법인은15%)이상변동
고유값 20240305901186
요청일자 2024.03.05
########################################
제출시간 18:58
시장 코스닥시장
회사이름 씨씨에스
회사 코드 00249441
공시이름 주요사항보고서(소송등의제기)
고유값 20240305000637
요청일자 2024.03.06
########################################
제출시간 18:50
시장 코스닥시장
회사이름 에스앤디
회사 코드 00560849
공시이름 주주총회집중일개최사유신고
고유값 20240305901177
요청일자 2024.03.05
########################################
제출시간 18:48
시장 코스닥시장
회사이름 씨유메디칼
회사 코드 00681249
공시이름 주주총회소집결의
고유값 20240305901172
요청일자 2024.03.05
########################################
제출시간 18:43
시장 유가증권시장
회사이름 남양유업
회사 코드 00107598
공시이름 소송등의제기ㆍ신청(경영권분쟁소송)
고유값 20240305801171
요청일자 2024.03.05
########################################
제출시간 18:43
시장 코스닥시장
회사이름 씨씨에스
회사 코드 00249441
  (이사회결의 효력정지 및 대표이사직무집행정지 가처분)
고유값 20240305901175
요청일자 2024.03.05
########################################
제출시간 18:42
시장 유가증권시장
회사이름 카프로
회사 코드 00159810
공시이름 주주총회소집결의
고유값 20240305801174
요청일자 2024.03.05
##########################

In [216]:
# Q2) 1번에서 출력한 내용을 df에 저장해보세요. 컬럼 이름은 스스로 정해보기

lst= []
cnt= len(soup.find("tbody").find_all("tr"))
         
for idx in range(cnt):
    temp = soup.find("tbody").find_all("tr")[idx]
    sub_time = temp.find_all("td")[0].text.strip()
    market = temp.find_all("td")[1].find_all("span")[1].get("title")
    cName = temp.find_all("td")[3].text
    code = re.findall(r"\d{8}",temp.find_all("td")[1].find("a").get("href"))[0]
    tName= re.sub(r"\t|\n","",temp.find_all("td")[2].text.strip())
    rcpNo= re.findall(r"\d{14}",temp.find_all("td")[2].find("a").get("href"))[0]
    ord_date = temp.find_all("td")[4].text

    lst.append([sub_time, market, cName, code, tName, rcpNo, ord_date])

df = pd.DataFrame(data= lst,
                columns = ["sub_time","market","cName","code","tName", "rcpNo","ord_date"]
                 )

In [218]:
df.head(4)

,sub_time,market,cName,code,tName,rcpNo,ord_date
0,19:29,코스닥시장,에이프릴바이오,01375822,[기재정정]매출액또는손익구조30%(대규모법인은15%)이상변동,20240305901186,2024.03.05
1,18:58,코스닥시장,씨씨에스,00249441,주요사항보고서(소송등의제기),20240305000637,2024.03.06
2,18:50,코스닥시장,에스앤디,00560849,주주총회집중일개최사유신고,20240305901177,2024.03.05
3,18:48,코스닥시장,씨유메디칼,00681249,주주총회소집결의,20240305901172,2024.03.05


In [229]:
# Q3) 2024년 3월 5일의 7페이지를 다 돌아서 
# 698의 공시 정보에 대한 내용에 대한 추출한 정보들을 1개의 df에 담아보세요.
date= "2024.03.05"

for pages in range(1,8):
    url = f"https://dart.fss.or.kr/dsac001/mainAll.do?selectDate={date}&sort=&series=&mdayCnt=0&currentPage={pages}"
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")

    lst= []
    cnt= len(soup.find("tbody").find_all("tr"))
             
    for idx in range(cnt):
        temp = soup.find("tbody").find_all("tr")[idx]
        sub_time = temp.find_all("td")[0].text.strip()
        market = temp.find_all("td")[1].find_all("span")[1].get("title")
        cName = temp.find_all("td")[3].text
        code = re.findall(r"\d{8}",temp.find_all("td")[1].find("a").get("href"))[0]
        tName= re.sub(r"\t|\n","",temp.find_all("td")[2].text.strip())
        rcpNo= re.findall(r"\d{14}",temp.find_all("td")[2].find("a").get("href"))[0]
        ord_date = temp.find_all("td")[4].text
    
        lst.append([sub_time, market, cName, code, tName, rcpNo, ord_date])

    if pages ==1:
        df = pd.DataFrame(data= lst,
                        columns = ["sub_time","market","cName","code","tName", "rcpNo","ord_date"]
                         )
    else:
        df1 = pd.DataFrame(data= lst,
                        columns = ["sub_time","market","cName","code","tName", "rcpNo","ord_date"]
                         )
        df = pd.concat([df,df1])

In [230]:
df

,sub_time,market,cName,code,tName,rcpNo,ord_date
0,19:29,코스닥시장,에이프릴바이오,01375822,[기재정정]매출액또는손익구조30%(대규모법인은15%)이상변동,20240305901186,2024.03.05
1,18:58,코스닥시장,씨씨에스,00249441,주요사항보고서(소송등의제기),20240305000637,2024.03.06
2,18:50,코스닥시장,에스앤디,00560849,주주총회집중일개최사유신고,20240305901177,2024.03.05
3,18:48,코스닥시장,씨유메디칼,00681249,주주총회소집결의,20240305901172,2024.03.05
4,18:43,유가증권시장,남양유업,00107598,소송등의제기ㆍ신청(경영권분쟁소송),20240305801171,2024.03.05
...,...,...,...,...,...,...,...
93,07:30,기타법인,식품안전상생재단,01604423,공시대상기업집단의특수관계인으로부터수증,20240304000588,2024.03.05
94,07:30,코스닥시장,아스트,00409681,주요사항보고서(전환사채권발행결정),20240304000592,2024.03.05
95,07:30,코스닥시장,엘컴텍,00242712,의결권대리행사권유참고서류,20240304000585,2024.03.05
96,07:30,기타법인,우란문화재단,01678589,공시대상기업집단의특수관계인으로부터수증,20240304000573,2024.03.05


In [235]:
# Q4) 이것을 함수로 만들어보기
# 입력 : 조회일자, 종료일자
# 출력 : 입력 기간 동안의 모든 공시 처리
# df, csv, escel etc

def report(findDT, endDT):
    findDT = datetime.strptime(findDT,"%Y.%m.%d")
    endDT = datetime.strptime(endDT,"%Y.%m.%d")
    
    for date in range(findDT, endDT): # 날짜별
        for pages in range(1,8):     # 그 날자의 페이별로...
                
            url = f"https://dart.fss.or.kr/dsac001/mainAll.do?selectDate={date}&sort=&series=&mdayCnt=0&currentPage={pages}"
            res = requests.get(url)
            soup = BeautifulSoup(res.text, "html.parser")
        
            lst= []
            cnt= len(soup.find("tbody").find_all("tr"))
                     
            for idx in range(cnt): # 그 페이지에 공시벼로...
                temp = soup.find("tbody").find_all("tr")[idx]
                sub_time = temp.find_all("td")[0].text.strip()
                market = temp.find_all("td")[1].find_all("span")[1].get("title")
                cName = temp.find_all("td")[3].text
                code = re.findall(r"\d{8}",temp.find_all("td")[1].find("a").get("href"))[0]
                tName= re.sub(r"\t|\n","",temp.find_all("td")[2].text.strip())
                rcpNo= re.findall(r"\d{14}",temp.find_all("td")[2].find("a").get("href"))[0]
                ord_date = temp.find_all("td")[4].text
            
                lst.append([sub_time, market, cName, code, tName, rcpNo, ord_date])
        
            if pages ==1:
                df = pd.DataFrame(data= lst,
                                columns = ["sub_time","market","cName","code","tName", "rcpNo","ord_date"]
                                 )
            else:
                df1 = pd.DataFrame(data= lst,
                                columns = ["sub_time","market","cName","code","tName", "rcpNo","ord_date"]
                                 )
                df = pd.concat([df,df1])

    return df
    

TypeError: 'datetime.datetime' object cannot be interpreted as an integer

In [239]:
datetime.strptime("2024.03.05","%Y.%m.%d")

datetime.datetime(2024, 3, 5, 0, 0)

In [241]:
datetime.strptime("2024.03.09","%Y.%m.%d")

datetime.datetime(2024, 3, 9, 0, 0)

In [244]:
from datetime import date, timedelta

In [255]:
in_days = "2024.03.05"
out_days ="2024.03.09"

In [256]:
in_days_dt = datetime.strptime(in_days,"%Y.%m.%d")
out_days_dt = datetime.strptime(out_days,"%Y.%m.%d")
delta = timedelta(days=1)

In [273]:
diff_days = out_days_dt - in_days_dt + delta# 끝날도 초함해야 해서...
diff_days

datetime.timedelta(days=5)

In [279]:
days_list = [ ( in_days_dt +delta*(i)).strftime("%Y.%m.%d")  for i in range(diff_days.days)  ]
days_list

['2024.03.05', '2024.03.06', '2024.03.07', '2024.03.08', '2024.03.09']

In [280]:
for day in days_list:
    print(day)

2024.03.05
2024.03.06
2024.03.07
2024.03.08
2024.03.09


TypeError: 'datetime.timedelta' object is not iterable

In [246]:
delta = timedelta(days=1)

In [247]:
datetime.strptime("2024.03.05","%Y.%m.%d") + delta

datetime.datetime(2024, 3, 6, 0, 0)

In [248]:
datetime.strptime("2024.03.09","%Y.%m.%d") - datetime.strptime("2024.03.05","%Y.%m.%d")

datetime.timedelta(days=4)

In [ ]:
diff_days = datetime.strptime("2024.03.09","%Y.%m.%d") - datetime.strptime("2024.03.05","%Y.%m.%d")


In [252]:
a = datetime.strptime("2024.03.05","%Y.%m.%d") + timedelta(days=1)
a.strftime("%Y.%m.%d")

'2024.03.06'

In [243]:
for i in range(datetime.strptime("2024.03.05","%Y.%m.%d"), datetime.strptime("2024.03.09","%Y.%m.%d")):
    print(i)

TypeError: 'datetime.datetime' object cannot be interpreted as an integer